In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as pyplot
%matplotlib inline

In [3]:
# read in all words
words = open("../lec1_intro_language_modeling/names.txt").read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
len(words)

32033

In [7]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {s:i for i,s in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [8]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one ?
X, Y = [], []
for w in words[:5]:
    
    print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '----->', itos[ix])
        context = context[1:] + [ix] # crop and append . Ruling window
        
X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... -----> e
..e -----> m
.em -----> m
emm -----> a
mma -----> .
olivia
... -----> o
..o -----> l
.ol -----> i
oli -----> v
liv -----> i
ivi -----> a
via -----> .
ava
... -----> a
..a -----> v
.av -----> a
ava -----> .
isabella
... -----> i
..i -----> s
.is -----> a
isa -----> b
sab -----> e
abe -----> l
bel -----> l
ell -----> a
lla -----> .
sophia
... -----> s
..s -----> o
.so -----> p
sop -----> h
oph -----> i
phi -----> a
hia -----> .


In [10]:
C = torch.randn((27,2)) # matrix of embeddings initialized randomly

In [14]:
F.one_hot(torch.tensor(5),num_classes=27).float() @ C

tensor([-0.3308,  1.9680])

In [17]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [18]:
W1 = torch.randn((6, 100)) # 2-dim embedding * 3 context length
b1 = torch.randn(100)

In [22]:
emb[:,0,:].shape # all the embeddings of just the first "word" in X
torch.cat([emb[:,0,:],emb[:,1,:],emb[:,2,:]],dim=1).shape

torch.Size([32, 6])

In [31]:
b = torch.cat(torch.unbind(emb,dim=1), dim=1)

In [25]:
a = torch.arange(18)
a

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])

In [34]:
h = torch.tanh(emb.view(-1,6)  @ W1 + b1) # PyTorch will infer the correct missing dimension

In [37]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [38]:
logits = h @ W2 + b2

In [39]:
logits.shape

torch.Size([32, 27])

In [40]:
counts = logits.exp()

In [41]:
prob = counts / counts.sum(1, keepdim=True)

In [43]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(16.5350)

In [44]:
g = torch.Generator().manual_seed(2147483647) # to include determinism
C = torch.randn((27,2), generator=g) # matrix of embeddings initialized randomly
W1 = torch.randn((6, 100), generator=g) # 2-dim embedding * 3 context length
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [46]:
sum(n.nelement() for n in parameters)

3481

In [50]:
for p in parameters:
    p.requires_grad = True

In [53]:
for _ in range(1000):
    # forward pass
    emb = C[X]
    h = torch.tanh(emb.view(-1,6)  @ W1 + b1) # PyTorch will infer the correct missing dimension
    logits = h @ W2 + b2
    # counts = logits.exp()   # The next 3 lines are called Cross-entropy
    # prob = counts / counts.sum(1, keepdim=True)
    # loss = -prob[torch.arange(32), Y].log().mean()
    loss = F.cross_entropy(logits, Y)

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    # update
    for p in parameters:
        p.data += -0.1 * p.grad
print(loss.item())

0.25599104166030884


In [48]:
F.cross_entropy(logits, Y)

tensor(17.7697)